<a href="https://colab.research.google.com/github/draaronv/statistical_learning/blob/master/classication_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('googledrive')
install.packages('httpuv') 
library('googledrive') 
library('httpuv')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘Rcpp’, ‘promises’, ‘later’




In [ ]:
if (file.exists("/usr/local/lib/python3.7/dist-packages/google/colab/_ipython.py")) {
  install.packages("R.utils")
  library("R.utils")
  library("httr")
  my_check <- function() {return(TRUE)}
  reassignInPackage("is_interactive", pkgName = "httr", my_check) 
  options(rlang_interactive=TRUE)
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.1 (2020-08-26 16:20:06 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.24.0 (2020-08-26 16:11:58 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.11.0 (2021-09-26 08:30:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, inherits, isOpen, nullfile, parse,
    warnings




In [ ]:
x <- drive_get("~/statistical_learning/finalData.csv")
drive_download(x)





✔ The input `path` resolved to exactly 1 file.

File downloaded:

• finalData.csv <id: 1W1wazYCqs5Iiz6cAwFUjo9f3a3kaBxWU>

Saved locally as:

• finalData.csv



In [ ]:
rm(list = ls())

In [ ]:
install.packages("tree")
install.packages("Metrics")
install.packages("rsample")
install.packages("rpart")
install.packages("dplyr")
install.packages("rpart.plot")
install.packages("ipred")
install.packages("caret")
install.packages("caTools")
install.packages("broom")
install.packages("parallel")
install.packages("doParallel")
install.packages("FNN")
install.packages("e1071")
install.packages("glmnet")
install.packages("randomForest")
install.packages("ranger")
library(ranger)
library(randomForest)
library(glmnet)
library(e1071)
library(parallel)
library(FNN)
library(doParallel)
library(broom)
library(tree)
library(Metrics)
library(rsample)     # data splitting 
library(dplyr)       # data wrangling
library(rpart)       # performing regression trees
library(rpart.plot)  # plotting regression trees
library(ipred)       # bagging
library(caret)       # bagging
library(caTools)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘listenv’, ‘parallelly’, ‘future’, ‘globals’, ‘warp’, ‘furrr’, ‘slider’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘future.apply’, ‘progressr’, ‘numDeriv’, ‘SQUAREM’, ‘lava’, ‘prodlim’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘proxy’, ‘iterators’, ‘gower’, ‘hardhat’, ‘timeDate’, ‘e1071’, ‘foreach’, ‘ModelMetrics’, ‘

In [ ]:
cluster <- makeCluster(detectCores() - 1) # convention to leave 1 core for OS
registerDoParallel(cluster)

# Data Initilization

In [ ]:
data1 = read.csv("finalData.csv")

In [ ]:
data2=head(data1,100000)

In [ ]:
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.50)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

[1] 99917

'data.frame':	49958 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 82 51 17 42 37 17 56 31 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 1 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 

# Tree

In [ ]:
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 49958 76380 0 ( 0.7747508 0.1520677 0.0430562 0.0162737 0.0076664 0.0029825 0.0015413 0.0003403 0.0004003 0.0002202 0.0005204 0.0001802 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8    10    12    18
  0  38631     0     0     0     0     0     0     0     0     0     0     0
  1   7613     0     0     0     0     0     0     0     0     0     0     0
  2   2124     0     0     0     0     0     0     0     0     0     0     0
  3    832     0     0     0     0     0     0     0     0     0     0     0
  4    423     0     0     0     0     0     0     0     0     0     0     0
  5    162     0     0     0     0     0     0     0     0     0     0     0
  6     88     0     0     0     0     0     0     0     0     0     0     0
  7     17     0     0     0     0     0     0     0     0     0     0     0
  8     19     0     0     0     0     0     0     0     0     0     0     0
  10    12     0     0     0     0     0     0     0     0     0     0     0
  12    28     0     0     0     0     0     0     0     0     0     0     0
  18    10     0     0     0    

# Testing Different Sizes

In [ ]:
data2=head(data1,200000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 200000

[1] 199917

[1] 199917

[1] 199917

'data.frame':	157077 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 15 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 157077 237600 0 ( 7.810e-01 1.456e-01 4.197e-02 1.789e-02 6.869e-03 3.400e-03 1.674e-03 4.393e-04 3.883e-04 3.183e-05 3.056e-04 1.273e-05 2.738e-04 8.913e-05 9.549e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  33501     0     0     0     0     0     0     0     0     0     0     0
  1   6199     0     0     0     0     0     0     0     0     0     0     0
  2   1739     0     0     0     0     0     0     0     0     0     0     0
  3    769     0     0     0     0     0     0     0     0     0     0     0
  4    322     0     0     0     0     0     0     0     0     0     0     0
  5    170     0     0     0     0     0     0     0     0     0     0     0
  6     65     0     0     0     0     0     0     0     0     0     0     0
  7     24     0     0     0     0     0     0     0     0     0     0     0
  8     18     0     0     0     0     0     0     0     0     0     0     0
  9      0     0     0     0     0     0     0     0     0     0     0     0
  10    12     0     0     0     0     0     0     0     0     0     0     0
  11     1     0     0     0    

In [ ]:
data2=head(data1,300000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                173                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 300000

[1] 299827

[1] 299826

[1] 299826

'data.frame':	235578 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 15 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 235578 355500 0 ( 7.822e-01 1.445e-01 4.186e-02 1.765e-02 7.025e-03 3.404e-03 1.745e-03 3.863e-04 4.372e-04 1.656e-04 2.589e-04 7.641e-05 1.825e-04 5.943e-05 6.367e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  50292     0     0     0     0     0     0     0     0     0     0     0
  1   9196     0     0     0     0     0     0     0     0     0     0     0
  2   2649     0     0     0     0     0     0     0     0     0     0     0
  3   1158     0     0     0     0     0     0     0     0     0     0     0
  4    468     0     0     0     0     0     0     0     0     0     0     0
  5    260     0     0     0     0     0     0     0     0     0     0     0
  6    112     0     0     0     0     0     0     0     0     0     0     0
  7     32     0     0     0     0     0     0     0     0     0     0     0
  8     29     0     0     0     0     0     0     0     0     0     0     0
  9      9     0     0     0     0     0     0     0     0     0     0     0
  10    18     0     0     0     0     0     0     0     0     0     0     0
  11     5     0     0     0    

In [ ]:
data2=head(data1,400000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                317                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 400000

[1] 399683

[1] 399682

[1] 399682

'data.frame':	314035 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 16 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 314035 473800 0 ( 7.818e-01 1.448e-01 4.251e-02 1.745e-02 6.824e-03 3.398e-03 1.614e-03 4.458e-04 4.394e-04 1.401e-04 2.197e-04 5.732e-05 1.751e-04 4.777e-05 4.458e-05 4.777e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  67041     0     0     0     0     0     0     0     0     0     0     0
  1  12256     0     0     0     0     0     0     0     0     0     0     0
  2   3635     0     0     0     0     0     0     0     0     0     0     0
  3   1523     0     0     0     0     0     0     0     0     0     0     0
  4    594     0     0     0     0     0     0     0     0     0     0     0
  5    323     0     0     0     0     0     0     0     0     0     0     0
  6    132     0     0     0     0     0     0     0     0     0     0     0
  7     40     0     0     0     0     0     0     0     0     0     0     0
  8     37     0     0     0     0     0     0     0     0     0     0     0
  9     12     0     0     0     0     0     0     0     0     0     0     0
  10    21     0     0     0     0     0     0     0     0     0     0     0
  11     5     0     0     0    

In [ ]:
data2=head(data1,500000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499557

[1] 499557

'data.frame':	392508 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 16 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 392508 589300 0 ( 7.824e-01 1.453e-01 4.219e-02 1.714e-02 6.713e-03 3.154e-03 1.518e-03 4.764e-04 4.815e-04 1.401e-04 1.758e-04 5.860e-05 1.401e-04 6.115e-05 3.567e-05 7.643e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  83893     0     0     0     0     0     0     0     0     0     0     0
  1  15383     0     0     0     0     0     0     0     0     0     0     0
  2   4477     0     0     0     0     0     0     0     0     0     0     0
  3   1865     0     0     0     0     0     0     0     0     0     0     0
  4    732     0     0     0     0     0     0     0     0     0     0     0
  5    367     0     0     0     0     0     0     0     0     0     0     0
  6    156     0     0     0     0     0     0     0     0     0     0     0
  7     51     0     0     0     0     0     0     0     0     0     0     0
  8     51     0     0     0     0     0     0     0     0     0     0     0
  9     14     0     0     0     0     0     0     0     0     0     0     0
  10    21     0     0     0     0     0     0     0     0     0     0     0
  11     7     0     0     0    

In [ ]:
data2=head(data1,600000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  2                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 600000

[1] 599558

[1] 599556

[1] 599556

'data.frame':	471079 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 17 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 471079 707900 0 ( 7.821e-01 1.455e-01 4.253e-02 1.681e-02 6.729e-03 3.186e-03 1.465e-03 4.904e-04 4.904e-04 1.401e-04 1.635e-04 7.005e-05 1.613e-04 5.095e-05 2.972e-05 6.368e-05 4.246e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  100624      0      0      0      0      0      0      0      0      0
  1   18517      0      0      0      0      0      0      0      0      0
  2    5421      0      0      0      0      0      0      0      0      0
  3    2195      0      0      0      0      0      0      0      0      0
  4     859      0      0      0      0      0      0      0      0      0
  5     460      0      0      0      0      0      0      0      0      0
  6     182      0      0      0      0      0      0      0      0      0
  7      60      0      0      0      0      0      0      0      0      0
  8      61      0      0      0      0      0      0      0      0      0
  9      22      0      0      0      0      0      0      0      0      0
  10     24      0      0      0      0      0      0      0      0      0
  11      9      0      0      0      0      0      0   

In [ ]:
data2=head(data1,700000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  2                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                815                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 700000

[1] 699185

[1] 699183

[1] 699183

'data.frame':	549357 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 18 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 549357 827400 0 ( 7.816e-01 1.458e-01 4.235e-02 1.672e-02 6.852e-03 3.369e-03 1.544e-03 5.661e-04 4.387e-04 1.420e-04 1.402e-04 8.009e-05 1.438e-04 4.369e-05 2.184e-05 2.548e-05 5.461e-05 4.369e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  117315      0      0      0      0      0      0      0      0      0
  1   21694      0      0      0      0      0      0      0      0      0
  2    6195      0      0      0      0      0      0      0      0      0
  3    2560      0      0      0      0      0      0      0      0      0
  4    1023      0      0      0      0      0      0      0      0      0
  5     552      0      0      0      0      0      0      0      0      0
  6     224      0      0      0      0      0      0      0      0      0
  7      83      0      0      0      0      0      0      0      0      0
  8      64      0      0      0      0      0      0      0      0      0
  9      27      0      0      0      0      0      0      0      0      0
  10     24      0      0      0      0      0      0      0      0      0
  11     12      0      0      0      0      0      0   

In [ ]:
data2=head(data1,800000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  4                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                933                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 800000

[1] 799067

[1] 799063

[1] 799063

'data.frame':	627835 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 18 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 627835 943500 0 ( 7.821e-01 1.456e-01 4.218e-02 1.677e-02 6.823e-03 3.394e-03 1.492e-03 5.607e-04 3.934e-04 1.242e-04 1.370e-04 8.123e-05 1.418e-04 3.823e-05 1.911e-05 4.778e-05 4.778e-05 3.823e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  134098      0      0      0      0      0      0      0      0      0
  1   24804      0      0      0      0      0      0      0      0      0
  2    7062      0      0      0      0      0      0      0      0      0
  3    2928      0      0      0      0      0      0      0      0      0
  4    1163      0      0      0      0      0      0      0      0      0
  5     625      0      0      0      0      0      0      0      0      0
  6     257      0      0      0      0      0      0      0      0      0
  7      90      0      0      0      0      0      0      0      0      0
  8      69      0      0      0      0      0      0      0      0      0
  9      27      0      0      0      0      0      0      0      0      0
  10     28      0      0      0      0      0      0      0      0      0
  11     15      0      0      0      0      0      0   

In [ ]:
data2=head(data1,900000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 900000

[1] 898894

[1] 898872

[1] 898872

'data.frame':	706257 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 19 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 706257 1060000 0 ( 7.824e-01 1.455e-01 4.209e-02 1.672e-02 6.796e-03 3.357e-03 1.475e-03 5.281e-04 4.007e-04 1.175e-04 1.373e-04 7.221e-05 1.260e-04 4.956e-05 1.699e-05 4.248e-05 2.832e-05 4.248e-05 3.398e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  150835      0      0      0      0      0      0      0      0      0
  1   27936      0      0      0      0      0      0      0      0      0
  2    7917      0      0      0      0      0      0      0      0      0
  3    3282      0      0      0      0      0      0      0      0      0
  4    1324      0      0      0      0      0      0      0      0      0
  5     696      0      0      0      0      0      0      0      0      0
  6     302      0      0      0      0      0      0      0      0      0
  7      95      0      0      0      0      0      0      0      0      0
  8      84      0      0      0      0      0      0      0      0      0
  9      29      0      0      0      0      0      0      0      0      0
  10     30      0      0      0      0      0      0      0      0      0
  11     15      0      0      0      0      0      0   

In [ ]:
data2=head(data1,1000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998872

[1] 998872

'data.frame':	784828 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 19 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 784828 1176000 0 ( 7.828e-01 1.454e-01 4.176e-02 1.671e-02 6.880e-03 3.284e-03 1.504e-03 5.632e-04 4.128e-04 1.058e-04 1.312e-04 7.772e-05 1.134e-04 4.460e-05 1.529e-05 3.822e-05 2.548e-05 3.822e-05 3.058e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  167642      0      0      0      0      0      0      0      0      0
  1   31003      0      0      0      0      0      0      0      0      0
  2    8805      0      0      0      0      0      0      0      0      0
  3    3615      0      0      0      0      0      0      0      0      0
  4    1515      0      0      0      0      0      0      0      0      0
  5     766      0      0      0      0      0      0      0      0      0
  6     349      0      0      0      0      0      0      0      0      0
  7     110      0      0      0      0      0      0      0      0      0
  8      91      0      0      0      0      0      0      0      0      0
  9      29      0      0      0      0      0      0      0      0      0
  10     33      0      0      0      0      0      0      0      0      0
  11     16      0      0      0      0      0      0   

In [ ]:
data2=head(data1,2000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 36                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2292                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2000000

[1] 1997708

[1] 1997672

[1] 1997672

'data.frame':	1569599 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 1569599 2407000 0 ( 7.758e-01 1.492e-01 4.382e-02 1.738e-02 7.338e-03 3.367e-03 1.439e-03 6.785e-04 3.122e-04 1.433e-04 1.567e-04 6.690e-05 7.645e-05 4.778e-05 1.529e-05 1.656e-05 3.377e-05 2.676e-05 2.294e-05 1.656e-05 1.529e-05 1.083e-05 1.274e-06 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  331946      0      0      0      0      0      0      0      0      0
  1   64070      0      0      0      0      0      0      0      0      0
  2   18639      0      0      0      0      0      0      0      0      0
  3    7341      0      0      0      0      0      0      0      0      0
  4    3190      0      0      0      0      0      0      0      0      0
  5    1537      0      0      0      0      0      0      0      0      0
  6     673      0      0      0      0      0      0      0      0      0
  7     251      0      0      0      0      0      0      0      0      0
  8     129      0      0      0      0      0      0      0      0      0
  9      71      0      0      0      0      0      0      0      0      0
  10     73      0      0      0      0      0      0      0      0      0
  11     25      0      0      0      0      0      0   

In [ ]:
data2=data1
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 39                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2737                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2529009

[1] 2526272

[1] 2526233

[1] 2526233

'data.frame':	1984898 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 1984898 3301000 0 ( 7.451e-01 1.702e-01 4.996e-02 1.964e-02 8.210e-03 3.619e-03 1.510e-03 7.507e-04 3.486e-04 1.592e-04 1.753e-04 7.356e-05 6.046e-05 4.282e-05 1.914e-05 1.914e-05 3.476e-05 2.821e-05 2.267e-05 1.310e-05 1.612e-05 8.565e-06 1.008e-06 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  403017      0      0      0      0      0      0      0      0      0
  1   92558      0      0      0      0      0      0      0      0      0
  2   26861      0      0      0      0      0      0      0      0      0
  3   10575      0      0      0      0      0      0      0      0      0
  4    4483      0      0      0      0      0      0      0      0      0
  5    2021      0      0      0      0      0      0      0      0      0
  6     893      0      0      0      0      0      0      0      0      0
  7     367      0      0      0      0      0      0      0      0      0
  8     176      0      0      0      0      0      0      0      0      0
  9      94      0      0      0      0      0      0      0      0      0
  10    102      0      0      0      0      0      0      0      0      0
  11     38      0      0      0      0      0      0   

# Adding Latitude and Longitude

In [ ]:
data2=data1
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
           LATITUDE           LONGITUDE      KILLED_INJURED             WEEKDAY 
                  0                   0                  39                   0 
            Holiday                temp          visibility            pressure 
                  0                   0                2737                   0 
           humidity          wind_speed        weather_main weather_description 
                  0                   0                   0                   0

[1] 2529009

[1] 2526272

[1] 2526233

[1] 2526233

'data.frame':	1894674 obs. of  16 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 33 82 42 51 51 17 42 37 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 1 2 2 2 2 2 2 2 2 2 ...
 $ LATITUDE           : num  40.7 40.7 40.7 40.7 40.7 ...
 $ LONGITUDE          : num  -73.9 -73.9 -73.9 -73.9 -73.9 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : n

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 1894674 3152000 0 ( 7.449e-01 1.705e-01 4.983e-02 1.962e-02 8.207e-03 3.673e-03 1.544e-03 7.257e-04 3.399e-04 1.626e-04 1.726e-04 6.967e-05 5.964e-05 4.275e-05 1.847e-05 1.900e-05 3.483e-05 2.850e-05 2.375e-05 1.372e-05 1.689e-05 8.973e-06 5.278e-07 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true      0      1      2      3      4      5      6      7      8      9
  0  470557      0      0      0      0      0      0      0      0      0
  1  107409      0      0      0      0      0      0      0      0      0
  2   31614      0      0      0      0      0      0      0      0      0
  3   12384      0      0      0      0      0      0      0      0      0
  4    5230      0      0      0      0      0      0      0      0      0
  5    2245      0      0      0      0      0      0      0      0      0
  6     965      0      0      0      0      0      0      0      0      0
  7     482      0      0      0      0      0      0      0      0      0
  8     224      0      0      0      0      0      0      0      0      0
  9     102      0      0      0      0      0      0      0      0      0
  10    123      0      0      0      0      0      0      0      0      0
  11     52      0      0      0      0      0      0   

# Downsample

In [ ]:
data2=head(data1,100000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  0                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                 83                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 100000

[1] 99917

[1] 99917

[1] 99917

...

'data.frame':	29693 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 12 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 10 levels "Clear","Clouds",..: 1 1 1 1 1 

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 29693 76800 1 ( 0.4017445 0.4026201 0.1133601 0.0439161 0.0212508 0.0081837 0.0043781 0.0008419 0.0011787 0.0006736 0.0013471 0.0005052 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true    0    1    2    3    4    5    6    7    8   10   12   18
  0     0 3281    0    0    0    0    0    0    0    0    0    0
  1     0 3255    0    0    0    0    0    0    0    0    0    0
  2     0  909    0    0    0    0    0    0    0    0    0    0
  3     0  341    0    0    0    0    0    0    0    0    0    0
  4     0  175    0    0    0    0    0    0    0    0    0    0
  5     0   68    0    0    0    0    0    0    0    0    0    0
  6     0   35    0    0    0    0    0    0    0    0    0    0
  7     0    9    0    0    0    0    0    0    0    0    0    0
  8     0    4    0    0    0    0    0    0    0    0    0    0
  10    0    3    0    0    0    0    0    0    0    0    0    0
  12    0   14    0    0    0    0    0    0    0    0    0    0
  18    0    4    0    0    0    0    0    0    0    0    0    0

Overall Statistics
                                          
               Accuracy : 0.402           
     

In [ ]:
data2=head(data1,500000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                  1                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
                442                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 500000

[1] 499558

[1] 499557

[1] 499557

...

'data.frame':	142188 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 33 42 39 38 35 39 35 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 16 levels "0","1","2","3",..: 1 1 1 4 4 4 4 4 2 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 142188 370000 1 ( 3.991e-01 4.003e-01 1.166e-01 4.755e-02 1.866e-02 8.967e-03 4.142e-03 1.315e-03 1.336e-03 3.868e-04 5.204e-04 1.829e-04 4.009e-04 1.407e-04 9.846e-05 2.040e-04 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0      0 15641     0     0     0     0     0     0     0     0     0     0
  1      0 15473     0     0     0     0     0     0     0     0     0     0
  2      0  4455     0     0     0     0     0     0     0     0     0     0
  3      0  1832     0     0     0     0     0     0     0     0     0     0
  4      0   714     0     0     0     0     0     0     0     0     0     0
  5      0   330     0     0     0     0     0     0     0     0     0     0
  6      0   163     0     0     0     0     0     0     0     0     0     0
  7      0    51     0     0     0     0     0     0     0     0     0     0
  8      0    50     0     0     0     0     0     0     0     0     0     0
  9      0    14     0     0     0     0     0     0     0     0     0     0
  10     0    16     0     0     0     0     0     0     0     0     0     0
  11     0     4     0     0    

In [ ]:
data2=head(data1,1000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 22                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               1106                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 1000000

[1] 998894

[1] 998872

[1] 998872

...

'data.frame':	284407 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  42 42 39 27 39 35 35 35 25 43 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 1 1 ...
 $ KILLED_INJURED     : Factor w/ 19 levels "0","1","2","3",..: 1 1 4 4 4 4 2 2 2 1 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 284407 738100 0 ( 4.015e-01 4.006e-01 1.150e-01 4.575e-02 1.923e-02 9.184e-03 4.290e-03 1.505e-03 1.097e-03 3.129e-04 3.938e-04 2.145e-04 3.235e-04 1.266e-04 4.219e-05 1.195e-04 7.384e-05 1.090e-04 9.493e-05 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  30932     0     0     0     0     0     0     0     0     0     0     0
  1  31215     0     0     0     0     0     0     0     0     0     0     0
  2   8863     0     0     0     0     0     0     0     0     0     0     0
  3   3721     0     0     0     0     0     0     0     0     0     0     0
  4   1447     0     0     0     0     0     0     0     0     0     0     0
  5    731     0     0     0     0     0     0     0     0     0     0     0
  6    309     0     0     0     0     0     0     0     0     0     0     0
  7    124     0     0     0     0     0     0     0     0     0     0     0
  8    103     0     0     0     0     0     0     0     0     0     0     0
  9     23     0     0     0     0     0     0     0     0     0     0     0
  10    24     0     0     0     0     0     0     0     0     0     0     0
  11    16     0     0     0    

In [ ]:
data2=head(data1,2000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 36                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2292                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2000000

[1] 1997708

[1] 1997672

[1] 1997672

...

'data.frame':	586464 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 51 37 27 38 35 39 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 4 4 4 4 4 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 586464 1527000 1 ( 3.995e-01 3.996e-01 1.172e-01 4.654e-02 1.985e-02 9.099e-03 3.927e-03 1.775e-03 8.287e-04 3.785e-04 4.314e-04 1.722e-04 2.029e-04 1.279e-04 4.092e-05 4.604e-05 9.037e-05 7.332e-05 6.309e-05 4.604e-05 4.604e-05 2.899e-05 0.000e+00 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0      0 64042     0     0     0     0     0     0     0     0     0     0
  1      0 63993     0     0     0     0     0     0     0     0     0     0
  2      0 18699     0     0     0     0     0     0     0     0     0     0
  3      0  7328     0     0     0     0     0     0     0     0     0     0
  4      0  3068     0     0     0     0     0     0     0     0     0     0
  5      0  1486     0     0     0     0     0     0     0     0     0     0
  6      0   628     0     0     0     0     0     0     0     0     0     0
  7      0   275     0     0     0     0     0     0     0     0     0     0
  8      0   133     0     0     0     0     0     0     0     0     0     0
  9      0    74     0     0     0     0     0     0     0     0     0     0
  10     0    66     0     0     0     0     0     0     0     0     0     0
  11     0    29     0     0    

In [ ]:
data2=data1
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,LATITUDE,LONGITUDE,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.90)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
     KILLED_INJURED             WEEKDAY             Holiday                temp 
                 39                   0                   0                   0 
         visibility            pressure            humidity          wind_speed 
               2737                   0                   0                   0 
       weather_main weather_description 
                  0                   0

[1] 2529009

[1] 2526272

[1] 2526233

[1] 2526233

...

'data.frame':	921103 obs. of  14 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  82 42 51 37 65 35 27 38 35 39 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 1 2 2 2 2 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 1 4 4 4 4 4 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : num  2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 2.1 ...
 $ weather_main       : Factor w/ 11 levels "Clear","Clouds",..: 1 1 1 1 1

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 921103 2383000 0 ( 4.006e-01 4.005e-01 1.174e-01 4.592e-02 1.931e-02 8.598e-03 3.615e-03 1.721e-03 8.001e-04 3.800e-04 4.191e-04 1.726e-04 1.400e-04 9.879e-05 4.343e-05 4.451e-05 8.251e-05 6.948e-05 5.211e-05 3.257e-05 4.234e-05 2.171e-05 1.086e-06 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  61372     0     0     0     0     0     0     0     0     0     0     0
  1  61501     0     0     0     0     0     0     0     0     0     0     0
  2  17902     0     0     0     0     0     0     0     0     0     0     0
  3   7251     0     0     0     0     0     0     0     0     0     0     0
  4   2996     0     0     0     0     0     0     0     0     0     0     0
  5   1284     0     0     0     0     0     0     0     0     0     0     0
  6    561     0     0     0     0     0     0     0     0     0     0     0
  7    272     0     0     0     0     0     0     0     0     0     0     0
  8    131     0     0     0     0     0     0     0     0     0     0     0
  9     60     0     0     0     0     0     0     0     0     0     0     0
  10    64     0     0     0     0     0     0     0     0     0     0     0
  11    25     0     0     0    

## Addig Latitude  and Longitude

In [ ]:
data2=head(data1,2000000)
## Data pre-processing
data2 = subset(data2, select = -c(CONTRIBUTING.FACTOR.VEHICLE.1,wind_deg,temp_min,temp_max,dew_point,CONTRIBUTING.FACTOR.VEHICLE.2,COLLISION_ID,X,VEHICLE.TYPE.CODE.1,VEHICLE.TYPE.CODE.2,ON.STREET.NAME,CROSS.STREET.NAME,OFF.STREET.NAME, VEHICLE_ID,weather_id,clouds_all,rain_1h,wind_gust,weather_icon,rain_3h,snow_3h,sea_level,grnd_level,snow_1h) )
names(data2)[names(data2) == "KILLED.INJURED"] <- "KILLED_INJURED"
colSums(is.na(data2))
data2$CRASH_DATE=as.Date(data2$CRASH_DATE, format = "%Y-%m-%d")
nrow(data2)
data2=data2[!is.na(data2$visibility),]
nrow(data2)
data2=data2[!is.na(data2$KILLED_INJURED),]
data2$KILLED_INJURED <- as.factor(data2$KILLED_INJURED)
nrow(data2)
index0 <- which(data2$KILLED_INJURED == 0)
index1 <- which(data2$KILLED_INJURED == 1)
deleteindex <- sample(index0,length(index0)-length(index1))
nrow(data2)
data2=data2[-deleteindex,]
nrow(data2)
data2$weather_main <- as.factor(data2$weather_main)
data2$weather_description <- as.factor(data2$weather_description)
data2$PERSON_SEX <- as.factor(data2$PERSON_SEX)
data2$WEEKDAY <- as.factor(data2$WEEKDAY)
set.seed(1)
sample = sample.split(data2, SplitRatio = 0.80)
training = subset(data2, sample == TRUE)
test = subset(data2, sample == FALSE)
rm(sample)
str(training)
str(test)
tree.brea=tree(KILLED_INJURED~., data=training)
tree.brea
pred=predict(tree.brea,test,type="class")
cm=table(true=test$KILLED_INJURED,predic=pred)
confusionMatrix(cm)

CRASH_DATE          CRASH_TIME          PERSON_AGE          PERSON_SEX 
                  0                   0                   0                   0 
           LATITUDE           LONGITUDE      KILLED_INJURED             WEEKDAY 
                  0                   0                  36                   0 
            Holiday                temp          visibility            pressure 
                  0                   0                2292                   0 
           humidity          wind_speed        weather_main weather_description 
                  0                   0                   0                   0

[1] 2000000

[1] 1997708

[1] 1997672

[1] 1997672

...

'data.frame':	559806 obs. of  16 variables:
 $ CRASH_DATE         : Date, format: "2019-10-25" "2019-10-25" ...
 $ CRASH_TIME         : int  15 15 15 15 15 15 15 15 15 15 ...
 $ PERSON_AGE         : num  33 82 51 37 27 38 35 39 35 35 ...
 $ PERSON_SEX         : Factor w/ 2 levels "F","M": 2 2 2 2 2 2 2 2 2 2 ...
 $ LATITUDE           : num  40.7 40.7 40.7 40.7 40.8 ...
 $ LONGITUDE          : num  -73.9 -73.9 -73.9 -73.9 -73.9 ...
 $ KILLED_INJURED     : Factor w/ 23 levels "0","1","2","3",..: 1 1 1 1 4 4 4 4 4 2 ...
 $ WEEKDAY            : Factor w/ 7 levels "Friday","Monday",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ Holiday            : chr  "False" "False" "False" "False" ...
 $ temp               : num  14.5 14.5 14.5 14.5 14.5 ...
 $ visibility         : num  10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ...
 $ pressure           : int  1027 1027 1027 1027 1027 1027 1027 1027 1027 1027 ...
 $ humidity           : int  63 63 63 63 63 63 63 63 63 63 ...
 $ wind_speed         : nu

Warning message in tree(KILLED_INJURED ~ ., data = training):
“NAs introduced by coercion”


node), split, n, deviance, yval, (yprob)
      * denotes terminal node

1) root 559806 1457000 0 ( 3.998e-01 3.994e-01 1.171e-01 4.648e-02 1.977e-02 9.064e-03 3.973e-03 1.749e-03 8.414e-04 4.037e-04 4.198e-04 1.804e-04 2.001e-04 1.268e-04 3.930e-05 4.823e-05 9.468e-05 7.860e-05 6.252e-05 4.466e-05 4.466e-05 3.037e-05 1.786e-06 ) *

Warning message in pred1.tree(object, tree.matrix(newdata)):
“NAs introduced by coercion”


Confusion Matrix and Statistics

    predic
true     0     1     2     3     4     5     6     7     8     9    10    11
  0  74498     0     0     0     0     0     0     0     0     0     0     0
  1  74753     0     0     0     0     0     0     0     0     0     0     0
  2  21846     0     0     0     0     0     0     0     0     0     0     0
  3   8600     0     0     0     0     0     0     0     0     0     0     0
  4   3642     0     0     0     0     0     0     0     0     0     0     0
  5   1748     0     0     0     0     0     0     0     0     0     0     0
  6    707     0     0     0     0     0     0     0     0     0     0     0
  7    337     0     0     0     0     0     0     0     0     0     0     0
  8    148     0     0     0     0     0     0     0     0     0     0     0
  9     70     0     0     0     0     0     0     0     0     0     0     0
  10    84     0     0     0     0     0     0     0     0     0     0     0
  11    29     0     0     0    